In [1]:
import pandas as pd
import prep_plays
pd.set_option('display.max_columns', None)

Acquire.py Loaded Successfully
Prep.py Loaded Successfully


In [2]:
df = prep_plays.prep_plays_data()

In [3]:
df.shape

(16925, 59)

In [4]:
df.isnull().sum()

playDescription           0
quarter                   0
down                      0
yardsToGo                 0
possessionTeam            0
offenseFormation          0
defendersInTheBox         0
numberOfPassRushers       0
QB_under_pressure         0
gameClock                 0
absoluteYardlineNumber    0
epa                       0
pass_stopped              0
playResult                0
RB                        0
TE                        0
WR                        0
DL                        0
LB                        0
DB                        0
ARI                       0
ATL                       0
BAL                       0
BUF                       0
CAR                       0
CHI                       0
CIN                       0
CLE                       0
DAL                       0
DEN                       0
DET                       0
GB                        0
HOU                       0
IND                       0
JAX                       0
KC                  

In [5]:
df.head()

,playDescription,quarter,down,yardsToGo,possessionTeam,offenseFormation,defendersInTheBox,numberOfPassRushers,QB_under_pressure,gameClock,absoluteYardlineNumber,epa,pass_stopped,playResult,RB,TE,WR,DL,LB,DB,ARI,ATL,BAL,BUF,CAR,CHI,CIN,CLE,DAL,DEN,DET,GB,HOU,IND,JAX,KC,LA,LAC,MIA,MIN,NE,NO,NYG,NYJ,OAK,PHI,PIT,SEA,SF,TB,TEN,WAS,EMPTY,I_FORM,JUMBO,PISTOL,SHOTGUN,SINGLEBACK,WILDCAT
0,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,ATL,I_FORM,7.0,4.0,0.0,15:00:00,90.0,0.261827,0,10,2,1,2,4,2,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,(13:10) M.Ryan pass incomplete short right to ...,1,1,10,ATL,SINGLEBACK,7.0,4.0,0.0,13:10:00,49.0,-0.372360,1,0,1,1,3,4,2,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,(13:05) (Shotgun) M.Ryan pass incomplete short...,1,2,10,ATL,SHOTGUN,6.0,4.0,0.0,13:05:00,49.0,-0.702779,1,0,2,1,2,4,2,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,(13:01) (Shotgun) M.Ryan pass deep left to J.J...,1,3,10,ATL,SHOTGUN,6.0,5.0,1.0,13:01:00,49.0,3.047530,0,33,1,1,3,4,1,6,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,(10:59) (Shotgun) M.Ryan pass incomplete short...,1,3,1,ATL,SHOTGUN,8.0,6.0,0.0,10:59:00,11.0,-0.842272,1,0,2,3,0,6,3,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [7]:
df.pass_stopped.value_counts()

0    11053
1     5872
Name: pass_stopped, dtype: int64

In [8]:
df.columns

Index(['playDescription', 'quarter', 'down', 'yardsToGo', 'possessionTeam',
       'offenseFormation', 'defendersInTheBox', 'numberOfPassRushers',
       'QB_under_pressure', 'gameClock', 'absoluteYardlineNumber', 'epa',
       'pass_stopped', 'playResult', 'RB', 'TE', 'WR', 'DL', 'LB', 'DB', 'ARI',
       'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL', 'DEN', 'DET',
       'GB', 'HOU', 'IND', 'JAX', 'KC', 'LA', 'LAC', 'MIA', 'MIN', 'NE', 'NO',
       'NYG', 'NYJ', 'OAK', 'PHI', 'PIT', 'SEA', 'SF', 'TB', 'TEN', 'WAS',
       'EMPTY', 'I_FORM', 'JUMBO', 'PISTOL', 'SHOTGUN', 'SINGLEBACK',
       'WILDCAT'],
      dtype='object')

In [11]:
pd.set_option('display.max_colwidth', -1)
df.playDescription.head()

<ipython-input-11-cf1e2b6f8cf9>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


0    (15:00) M.Ryan pass short right to J.Jones pushed ob at ATL 30 for 10 yards (M.Jenkins).
1    (13:10) M.Ryan pass incomplete short right to C.Ridley (J.Mills, J.Hicks).              
2    (13:05) (Shotgun) M.Ryan pass incomplete short left to D.Freeman.                       
3    (13:01) (Shotgun) M.Ryan pass deep left to J.Jones to PHI 6 for 33 yards (R.Darby).     
4    (10:59) (Shotgun) M.Ryan pass incomplete short right to D.Freeman.                      
Name: playDescription, dtype: object

In [12]:
df.numberOfPassRushers.value_counts()

4.0    10654
5.0    3245 
3.0    1517 
6.0    964  
2.0    245  
7.0    192  
1.0    72   
8.0    21   
0.0    14   
9.0    1    
Name: numberOfPassRushers, dtype: int64

In [13]:
df.defendersInTheBox.value_counts()

6.0     7896
7.0     3864
5.0     3043
4.0     1042
8.0     881 
3.0     118 
9.0     54  
2.0     14  
10.0    7   
11.0    4   
1.0     2   
Name: defendersInTheBox, dtype: int64